In [ ]:
#72%

In [1]:
import os
import pandas as pd
import pdfplumber
from transformers import pipeline

# Initialize the question-answering pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Function to extract diseases using question-answering
def extract_diseases(text):
    question = "What diseases are mentioned in the text?"
    answer = qa_pipeline(question=question, context=text, max_answer_len=150)
    diseases = set(answer['answer'].split(', '))
    return list(diseases)

# Read PDF files and create the pandas dataframe
folder_path = '/Users/amin/Desktop/33/test'  # Replace with your folder path
data = {'File': [], 'Issue_KW': []}

for file_name in os.listdir(folder_path):
    if file_name.endswith('.pdf'):
        file_path = os.path.join(folder_path, file_name)
        with pdfplumber.open(file_path) as pdf:
            text = ' '.join([page.extract_text() for page in pdf.pages])
        diseases = extract_diseases(text)
        data['File'].append(file_name[:-4])
        data['Issue_KW'].append(', '.join(diseases))

df = pd.DataFrame(data)
df.to_csv('Issue_KW.csv', index=False)


In [3]:
df.to_csv('df9.csv', index=False)
df

,File,Issue_KW
0,Zehna Corporate Deck SymBiosis,"urinalysis, blood analytes, plasma TMAO, blood..."
1,PS_TCRR_NOTE_1.5.23,ovarian cancer
2,Grace Sciece Platform (9.2.20),Neurodegenerative\ndiseases\nNGLY1 Deﬁciency\n...
3,AcuamarkDx Series A_Corp Deck,cancer
4,coding.bio_deck,cancers
5,Centurion_BioPharma-Non-Confidential_2021_2,"immunology, inflammation, allergy, and infecti..."
6,JPM_SRZN_NOTE_11.28.22,tissue damage mediated\ndisorders
7,G_ACET_IOC_4.8.21,"carcinoma, melanoma, and glioblastoma"
8,C_ARQT_NOTE_11.15.22,psoriasis and AD
9,B_RXRX_IOC_9.21.21,Tay-Sachs and Sandhoff Disease


In [4]:
import configparser
import pandas as pd
import openai
import re

def load_openai_api_key():
    config = configparser.ConfigParser()
    config.read("config.ini")
    return config["openai"]["api_key"]

def find_matching_area(answer):
    areas = [
        "Vaccines or infectious",
        "Gastrointestinal or metabolism",
        "Neurological",
        "Cancer or tumors or oncology",
        "Dermatological",
        "Organ health",
        "Mental",
        "Other",
    ]
    for area in areas:
        if re.search(area.lower(), answer.lower()):
            return area
    return "Other"

def get_related_area(keywords):
    openai.api_key = load_openai_api_key()
    prompt = f"Which of the following areas is more related to the keywords: {keywords}?\n\nAreas:\n" \
             "Vaccines or infectious\n" \
             "Gastrointestinal or metabolism\n" \
             "Neurological\n" \
             "Cancer or tumors or oncology\n" \
             "Dermatological\n" \
             "Organ health\n" \
             "Mental\n" \
             "Other\n\nAnswer: "

    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=10,
        n=1,
        stop=None,
        temperature=0.5,
    )

    answer = response.choices[0].text.strip()
    matching_area = find_matching_area(answer)
    return matching_area

def main():
    # Load your DataFrame
    df = pd.read_csv("df9.csv")

    # Add a new column 'Issue'
    df["Issue"] = df["Issue_KW"].apply(get_related_area)

    # Drop the 'KW' column
    df = df.drop(columns=["Issue_KW"])
    print(df)

    # Save the DataFrame to a new CSV file
    df.to_csv("issue9.csv", index=False)

if __name__ == "__main__":
    main()


                                                 File  \
0                      Zehna Corporate Deck SymBiosis   
1                                 PS_TCRR_NOTE_1.5.23   
2                      Grace Sciece Platform (9.2.20)   
3                       AcuamarkDx Series A_Corp Deck   
4                                     coding.bio_deck   
5         Centurion_BioPharma-Non-Confidential_2021_2   
6                              JPM_SRZN_NOTE_11.28.22   
7                                   G_ACET_IOC_4.8.21   
8                                C_ARQT_NOTE_11.15.22   
9                                  B_RXRX_IOC_9.21.21   
10               Dianomi Non-Confidential Slides - v3   
11                           KBC_MAAT.FR_IOC_12.14.21   
12                               Engrail Therapeutics   
13                                 WB_VOR_IOC_1.25.22   
14                                 E_FNCH_IOC_8.10.21   
15                                CG_GRNA_IOC_12.8.22   
16                             